In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -U sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=0f14a46c97d3080699c590ee397314042fe01ad9f8e8fb58c62f84898682cea2
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from sentence_transformers import SentenceTransformer, util, evaluation , LoggingHandler, losses, InputExample
import random
import json
from tqdm import tqdm
from sklearn.model_selection import train_test_split


In [4]:
warmup_sets = '/content/drive/MyDrive/pRoBERTa/data/converted_data_warmup_tocken.json'
train_sets = '/content/drive/MyDrive/pRoBERTa/data/converted_data_train_tocken.json'
save_model_path = '/content/drive/MyDrive/pRoBERTa/trained_module/labels_prediction/label_sbert'
model_name = "keepitreal/vietnamese-sbert"

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device "{device}"')

Using device "cuda"


# Load data

In [6]:
class LoadDataFromJson(Dataset):
  def __init__(self, datasets):
      self.samples = []

      with open(datasets) as file:
          data = json.load(file)

      for id, item in data.items():

          if item['evidence']:
              data_sample = {}
              data_sample['id'] = id
              data_sample['claim']  = item['claim']
              data_sample['evidence'] = item['evidence']
              data_sample['verdict'] = item['verdict']
              data_sample['domain'] = item['domain']
              self.samples.append(data_sample)

          # else:
          #   sentences = item["context"]
          #   random_sentence = random.sample(sentences, 3)
          #   for sentence in random_sentence:
          #     data_sample = {}
          #     data_sample['id'] = id
          #     data_sample['claim'], data_sample['evidence'] = delete_item(item['claim'], sentence)
          #     data_sample['verdict'] = 'NEI'
          #     data_sample['domain'] = item['domain']
          #     self.samples.append(data_sample)

  def to_dataframe(self):
      return pd.DataFrame(self.samples)

  def __len__(self):
      return len(self.samples)

  def __getitem__(self, idx):
      return self.samples[idx]

In [7]:
trainsets = LoadDataFromJson(train_sets)
df = trainsets.to_dataframe()
df = df.sample(frac=1, random_state=42)

In [8]:
df['verdict'].unique()

array(['SUPPORTED', 'REFUTED'], dtype=object)

In [9]:
df

,id,claim,evidence,verdict,domain
12417,38906,mà trong đời bạn nên thử ít_nhất một lần,mà bạn nên thử ít_nhất một lần trong đời,SUPPORTED,dulich
4394,11520,bài viết có nguồn đã chứng_thực,trưởng khoa khám tư_vấn dinh_dưỡng người_lớn v...,SUPPORTED,suckhoe
13326,46679,trước khi có khách chủ quán đã bắt_đầu nướng n...,khi có khách chủ quán mới bắt_đầu nướng những ...,REFUTED,dulich
12010,36719,mỗi nhà_hàng sẽ phục_vụ đồ_ăn kèm theo cách ri...,mỗi nhà_hàng sẽ phục_vụ đồ_ăn kèm khác nhau ch...,SUPPORTED,dulich
4653,11779,phong_tran có một con gái nhỏ,anh có một con gái nhỏ,SUPPORTED,thegioi
...,...,...,...,...,...
5191,12317,peter zhu nói rằng golden_lead chỉ là nhà_kinh...,chúng_tôi không phải ngư_dân mà chỉ là người k...,SUPPORTED,thegioi
13418,46771,thật_sự pháo_hoa giống như một đặc_sản của vũn...,thật_sự pháo_hoa giống như một đặc_sản của đà_...,REFUTED,dulich
5390,12516,tất_cả món tôi đã gọi đều rất ngon dù tôi chưa...,tôi chưa thử hết món trong thực_đơn nhưng tất_...,SUPPORTED,dulich
860,7985,chùm vhee được tạo ra từ loại máy_gia_tốc thẳn...,thí_nghiệm có_thể mở_đường cho việc lắp_đặt nh...,SUPPORTED,khoahoc


In [10]:
df_filter = df.loc[df['evidence'].apply(lambda x: len(x.split())) < 3]
df_filter

,id,claim,evidence,verdict,domain
13751,47104,các cô các chú phải ra_sức đem hiểu_biết khoa_...,hải_minh,REFUTED,khoahoc
1491,8616,thí_sinh phải đạt học_lực giỏi lớp 12 có chứng...,dương_tâm_lệ nguyễn,SUPPORTED,giaoduc
9778,30605,nhiều công_ty nợ 2 năm trở lên,hồng chiêu,SUPPORTED,thoisu
1957,9082,các vaccine kết_hợp 5 trong 1 ( pentaxim_pháp ...,anh chi,SUPPORTED,suckhoe
8726,20355,công_suất của nhà_máy nước thủ_đức mỗi ngày_đê...,gia_minh,SUPPORTED,thoisu
...,...,...,...,...,...
7892,19222,giá vé qua trạm trên quốc_lộ 1 cho 5 nhóm xe s...,hoàng nam,SUPPORTED,thoisu
13103,46456,các cô các chú không được đem hiểu_biết khoa_h...,hải_minh,REFUTED,khoahoc
11016,35725,thị_trấn cổ đã có 1300 tuổi và mang đến một vẻ...,tổng_hợp,SUPPORTED,dulich
8792,20421,không có nhà_trẻ mẫu_giáo nào trong 16 cụm côn...,hồng chiêu,SUPPORTED,thoisu


In [11]:
# Xóa những dòng có evidence < 3
df = df.drop(df[df.isin(df_filter.to_dict('list')).all(1)].index)

In [12]:
df.isna().sum()

id          0
claim       0
evidence    0
verdict     0
domain      0
dtype: int64

In [13]:
df.columns

Index(['id', 'claim', 'evidence', 'verdict', 'domain'], dtype='object')

In [14]:
len(df)

14050

In [15]:
df.loc[df['id'] == "47862"]

,id,claim,evidence,verdict,domain
14212,47862,3 h dj tắt_nhạc và thông_báo câu_lạc_bộ chuẩn_...,2 h dj tắt_nhạc và thông_báo câu_lạc_bộ chuẩn_...,REFUTED,dulich


In [16]:
df['verdict'] = df['verdict'].map(lambda x: 1 if x == "SUPPORTED" else 0)

In [17]:
df

,id,claim,evidence,verdict,domain
12417,38906,mà trong đời bạn nên thử ít_nhất một lần,mà bạn nên thử ít_nhất một lần trong đời,1,dulich
4394,11520,bài viết có nguồn đã chứng_thực,trưởng khoa khám tư_vấn dinh_dưỡng người_lớn v...,1,suckhoe
13326,46679,trước khi có khách chủ quán đã bắt_đầu nướng n...,khi có khách chủ quán mới bắt_đầu nướng những ...,0,dulich
12010,36719,mỗi nhà_hàng sẽ phục_vụ đồ_ăn kèm theo cách ri...,mỗi nhà_hàng sẽ phục_vụ đồ_ăn kèm khác nhau ch...,1,dulich
4653,11779,phong_tran có một con gái nhỏ,anh có một con gái nhỏ,1,thegioi
...,...,...,...,...,...
5191,12317,peter zhu nói rằng golden_lead chỉ là nhà_kinh...,chúng_tôi không phải ngư_dân mà chỉ là người k...,1,thegioi
13418,46771,thật_sự pháo_hoa giống như một đặc_sản của vũn...,thật_sự pháo_hoa giống như một đặc_sản của đà_...,0,dulich
5390,12516,tất_cả món tôi đã gọi đều rất ngon dù tôi chưa...,tôi chưa thử hết món trong thực_đơn nhưng tất_...,1,dulich
860,7985,chùm vhee được tạo ra từ loại máy_gia_tốc thẳn...,thí_nghiệm có_thể mở_đường cho việc lắp_đặt nh...,1,khoahoc


In [18]:
df['verdict'].value_counts()

1    12492
0     1558
Name: verdict, dtype: int64

In [19]:
df_eval_0 = df[df['verdict'] == 0].head(150) # supported
df_eval_1 = df[df['verdict'] == 1].head(150) # refuted
df_eval = pd.concat([df_eval_0, df_eval_1])


In [20]:
df_train = df[~df.isin(df_eval).all(axis=1)]
df_train

,id,claim,evidence,verdict,domain
11202,35911,tp phổ_yên rộng trên 6 ha tại phường ba hàng,phổ_yên ) có diện_tích trên 6 ha tại phường ba...,1,dulich
11301,36010,các nhà_nghiên cưu_đo tiếng vang của các loài ...,các nhà_nghiên_cứu sử_dụng đầu thu sóng dưới n...,1,khoahoc
3405,10531,theo giáo_sư george_gollin mỗi hạt di_chuyển đ...,điều này tạo ra những khu_vực có mật_độ cao hơ...,1,khoahoc
10963,35672,suy_đoán của nhóm nghiên_cứu được chứng_minh k...,suy_đoán của nhóm nghiên_cứu được khẳng_định k...,1,khoahoc
11502,36211,theo ước_tính của cơ_quan vũ_trụ châu_âu ( esa...,cơ_quan vũ_trụ châu_âu ( esa ) ước_tính vùng k...,1,khoahoc
...,...,...,...,...,...
5191,12317,peter zhu nói rằng golden_lead chỉ là nhà_kinh...,chúng_tôi không phải ngư_dân mà chỉ là người k...,1,thegioi
13418,46771,thật_sự pháo_hoa giống như một đặc_sản của vũn...,thật_sự pháo_hoa giống như một đặc_sản của đà_...,0,dulich
5390,12516,tất_cả món tôi đã gọi đều rất ngon dù tôi chưa...,tôi chưa thử hết món trong thực_đơn nhưng tất_...,1,dulich
860,7985,chùm vhee được tạo ra từ loại máy_gia_tốc thẳn...,thí_nghiệm có_thể mở_đường cho việc lắp_đặt nh...,1,khoahoc


In [21]:
df_train['verdict'].value_counts()

1    12342
0     1408
Name: verdict, dtype: int64

In [22]:
batch_size = 16
sampler = torch.utils.data.sampler.RandomSampler
# batch_sampler = torch.utils.data.sampler.BatchSampler

train_examples = []
for index, row in df_train.iterrows():
    claim = row['claim']
    sentence = row['evidence']
    verdict = row['verdict']
    input = InputExample(texts=[claim, sentence], label=verdict)
    train_examples.append(input)

train_dataloader = DataLoader(train_examples, batch_size=batch_size)

eval_examples = []
for index, row in df_eval.iterrows():
    claim = row['claim']
    sentence = row['evidence']
    verdict = row['verdict']
    input = InputExample(texts=[claim, sentence], label=verdict)
    eval_examples.append(input)

# Prepare model

In [23]:
# Load model from huggingface
model = SentenceTransformer(model_name)

In [24]:
sentences = ["ở vòng này độc giả vnexpress sẽ bình chọn cho sản phẩm / giải pháp mà mình yêu thích và đánh giá cao", "vì vậy sau khi trở lại sài gòn chị đã quyết định mở hàng ốc nhồi hơn một năm trước"]
embeddings = model.encode(sentences)
print(embeddings)
util.pytorch_cos_sim(embeddings[0], embeddings[1])

[[-0.09916721  0.28454182 -0.12842584 ...  0.3223491   0.14667584
  -0.11820575]
 [-0.01042098 -0.03861305  0.18633303 ...  0.33944547 -0.18970798
  -0.19059181]]


tensor([[0.1355]])

# Train

In [25]:
evaluator = evaluation.BinaryClassificationEvaluator.from_input_examples(eval_examples)

train_loss = losses.ContrastiveLoss(model=model)


model.fit([(train_dataloader, train_loss)],
          show_progress_bar = True,
          epochs=10,
          evaluation_steps = 1000,
          evaluator = evaluator,
          max_grad_norm = 1,
          optimizer_class = torch.optim.AdamW,
          optimizer_params = {"lr" : 2e-05},
          scheduler = "WarmupLinear",
          steps_per_epoch = None,
          warmup_steps = 144,
          weight_decay = 0.01,
          # checkpoint_path = save_checkpoint,
          output_path = save_model_path
          )


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/860 [00:00<?, ?it/s]

Iteration:   0%|          | 0/860 [00:00<?, ?it/s]

Iteration:   0%|          | 0/860 [00:00<?, ?it/s]

Iteration:   0%|          | 0/860 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/860 [00:00<?, ?it/s]

Iteration:   0%|          | 0/860 [00:00<?, ?it/s]

Iteration:   0%|          | 0/860 [00:00<?, ?it/s]

Iteration:   0%|          | 0/860 [00:00<?, ?it/s]

Iteration:   0%|          | 0/860 [00:00<?, ?it/s]

In [26]:
# Load model from huggingface
model_trained = SentenceTransformer(save_model_path)

In [34]:
claim = "trước khi có khách chủ quán đã bắt_đầu nướng n...	"
evidence = "khi có khách chủ quán mới bắt_đầu nướng những ...	"
sentences = []
sentences.append(claim)
sentences.append(sentences)

In [35]:
embeddings = model.encode(sentences)
print(embeddings)
print(util.pytorch_cos_sim(embeddings[0], embeddings[1]))

embeddings = model_trained.encode(sentences)
print(embeddings)
print(util.pytorch_cos_sim(embeddings[0], embeddings[1]))


[[-0.5958025   0.05960424  0.27427647 ...  0.32219502  0.18992656
   0.31643656]
 [-0.79022485  0.25167993  0.01293563 ...  0.46645403  0.37534037
   0.40537655]]
tensor([[0.9213]])
[[-0.641403    0.07844301  0.26853743 ...  0.3700302   0.21857275
   0.29849422]
 [-0.78805095  0.22983462  0.02667235 ...  0.493672    0.38098758
   0.36377513]]
tensor([[0.9385]])
